# Fitting data in Python 

After this short tutorial, you will be able to do the following things in Python:
* define a linear function
* fit a curve to some data using curve_fit

## Jupyter

Before we get started first some information about using Jupyter notebooks. The notebook consists of 
different blocks of python code which we call cells. Using the arrow keys you can select a cell.

Sometimes you will have to edit the code in the cells.
To edit the contents of a cell click somewhere in the code of the cell. To execute the code press <b>CTRL+Enter</b>.

<i>All keyboard shortcuts can be found by pressing the small keyboard in the menu bar. Many of these functions are also accessible using the mouse and the menu bar. Moving the cursor to a function and pressing shift+tab gives information about the function.</i>

## The actual program
Here we clear all variables and import some packages which we will use later on (select the cell below and press <b>CTRL+Enter</b> to run it). You do not need to understand this yet. You will learn what this means during your Python course in the first year or you can read the comments for some explanation or explore the accompanying websites: http://numpy.org/ & https://matplotlib.org/3.1.1/tutorials/introductory/pyplot.html & https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html.

In [ ]:
# Clear all variables

%clear

# One major feature of the IPython kernel is the ability to display plots that are the output of running code cells.
# The IPython kernel is designed to work seamlessly with the matplotlib plotting library to provide this functionality.
# To set this up, before any plotting or import of matplotlib is performed you must execute the %matplotlib magic command.
# This performs the necessary behind-the-scenes setup for IPython to work correctly hand in hand with matplotlib;
# It does not, however, actually execute any Python import commands, that is, no names are added to the namespace.

%matplotlib inline

# Numpy is the fundamental package for scientific computing with Python: http://numpy.org/
# Numpy provides support to mathematically manipulate large, multi-dimensional arrays and matrices.
# By importing this library in this way you can call all functions of numpy with the abbreviation np:
# For example, np.arange to place a series of numbers in an array.

import numpy as np

# Throughout your studies you will most likely use matplotlib.pyplot to plot any graph you need to
# illustrate your research: https://matplotlib.org/3.1.1/tutorials/introductory/pyplot.html
# matplotlib.pyplot is a collection of command style functions that make matplotlib work like MATLAB.
# Each pyplot function makes some change to a figure: e.g., creates a figure, creates a plotting area in a figure,
# plots some lines in a plotting area, decorates the plot with labels, etc.

import matplotlib.pyplot as plt

# We use the datetime library so we can easily add a datestamp to the names of the figures we are going to save.

from datetime import datetime

from scipy.optimize import curve_fit

# Now press CTRL+Enter to run this part of the code.

In the previous tutorials we have seen how to import data and how to plot those data:

In [ ]:
filename =  "201809Voorbeeldvervaltijdreelefout.csv"

mydata = np.loadtxt(filename, delimiter=",")
time=mydata[:,0]
R=mydata[:,1]
timeerror=mydata[:,2]
Rerror=mydata[:,3]

plt.figure()
plt.errorbar(time,R,xerr=timeerror,yerr=Rerror,capsize=2,elinewidth=0.5,fmt="o")
plt.xlabel("Time (s)")
plt.ylabel("Resistance (Ohm)")
plt.title('Resistance vs Time')
#plt.figtext(0,-0.05,'Figure #.. Plot of Resistance vs Time. Notice the possibly exponential decay.')

Often we have some expectation from theory about the relation between two quantities. For instance, the theory might predict an exponential relation between two quantities: \begin{equation*}I = I_0 \cdot e^{-t/t_0}\end{equation*} In an experiment, we often want to see two things. First, we want to know whether the theory does indeed describe the experiment. Secondly, we would like to know the values of the parameters in the theory. In this case the values for $I_0$ and $t_0$.

To this end, we try to create a curve given by the relation predicted by theory, which closely follows the experimental data points. This procedure is called fitting.

Before we can do the actual fit, we have to define a function that describes the theory. We have seen before that in python a function is something that accepts arguments and can return a value. Here we define our own function. The function accepts three variables and returns a value calculated with these variables.

To be able to see whether the function describes the experiment it is necessary to linearize your data because one can't tell whether a graph of the data is exactly exponential (or any other non-linear function), or not. However, one can tell by looking at a graph whether it is roughly linear or not.

To linearize your data, see section 0.9.2 in the lab manual. For example in the case of \begin{equation*}I = I_0 \cdot e^{-t/t_0}\end{equation*} one would not use $I$ and $t$ to do the curve_fit but instead we would use $ln(I)$ and $t$ which should have a linear relationship if the theory is correct. <i> One can check this by taking the natural logarithm of both sides of the equation: \begin{equation*}ln(I) = ln(I_0 \cdot e^{-t/t_0})\end{equation*}
\begin{equation*}ln(I) = -(1/t_0) \cdot t + ln(I_0)\end{equation*}
This function is linear in $t$ with $a = -(1/t_0)$ and $b = ln(I_0)$.

So to do a proper fit we will always use a linear function. We need to define that function first.

In [ ]:
# Here we define a function. We will always use a linear function.
# If your data is not linear, linearize it: see section 0.9.2 from the lab manual.
def linear_function(x,a,b):
    return a*x + b

# Here we test the new function
y=linear_function(1,2,3)
print(y)

To make a fit we use the function curve_fit, which we imported in the first cell. The arguments for curve_fit are the function we would like to fit to, the x variables and the y variables.

The function curve_fit returns an array with optimal parameters popt and an array with covariances pcov.

In [ ]:
# Here we calculate the optimal parameters
popt, pcov = curve_fit(linear_function, time, R,sigma=Rerror)

# Here we print the equation for the fitted curve
afit=popt[0]
bfit=popt[1]
print("yfit = "+str(afit)+" * t + "+str(bfit))

To see whether the fit makes any sense we can plot it together with the original curve.

In [ ]:
# Here we calculate the yvalues 
yfit = linear_function(time,afit,bfit)

plt.figure()
plt.errorbar(time,R,xerr=timeerror,yerr=Rerror,capsize=2,elinewidth=0.5,fmt="o")
plt.plot(time,yfit) #here we plot the fitted curve
plt.xlabel("Time (s)")
plt.ylabel("Resistance (Ohm)")

We also would like to know what the error on the fit parameters is. The curve_fit function also returned the variable pcov. This is the so called covariance matrix. On the diagonal of this matrix you can find the variance of the fit parameters. The variance is the square of the standard deviation. Therefore:

In [ ]:
variance_a = pcov[0,0]
variance_b = pcov[1,1]

sigma_a = np.sqrt(variance_a)
sigma_b = np.sqrt(variance_b)

print("f = a * x + b")
print("with")
print("a = "+str(afit)+" ± "+str(sigma_a))
print("b = "+str(bfit)+" ± "+str(sigma_b))

Even though the fit will come up with neat numbers for the fit, from the graph you can clearly tell that a linear fit does not work in this case.

We therefore need to manipulate the data (according to an underlying theory) to make the data linear. Since in this case we expect an exponential decay we can linearize the data by plotting $ln(R)$ against $Time$.

In [ ]:
# Manipulate the data and the accompanying errors to make it linear.

lnR = np.log(R)
lnRerror = (1/R)*Rerror

In [ ]:
# Here we calculate the optimal parameters
popt, pcov = curve_fit(linear_function, time, lnR,sigma=lnRerror)

# Here we print the equation for the fitted curve
afit=popt[0]
bfit=popt[1]
print("yfit = "+str(afit)+" * t + "+str(bfit))

In [ ]:
# Here we calculate the yvalues 
yfit = linear_function(time,afit,bfit)

plt.figure()
plt.errorbar(time,lnR,xerr=timeerror,yerr=lnRerror,capsize=2,elinewidth=0.5,fmt="o")
plt.plot(time,yfit) #here we plot the fitted curve
plt.xlabel("Time (s)")
plt.ylabel("ln(Resistance) (ln(Ohm)")

<b>Check whether the linear fit now neatly follows the data.</b>

In [ ]:
variance_a = pcov[0,0]
variance_b = pcov[1,1]

sigma_a = np.sqrt(variance_a)
sigma_b = np.sqrt(variance_b)

print("f = a * x + b")
print("with")
print("a = "+str(afit)+" ± "+str(sigma_a))
print("b = "+str(bfit)+" ± "+str(sigma_b))

<b>Note that the errors in the returned values should now be much smaller than before!</b>

Also note that python returns values with many insignificant numbers and without units. Make sure that you report the values in the correct significant numbers and units in your labjournal/report.<BR>
<i>The error should never have more than 1 significant digit and the calculated value should be rounded off to the position of that error digit.</i>

<b>Now finish this fitprocedure and calculate the values of $t_0$ and $I_0$ for this data and their errors!</b>

In [ ]:
# Your code here:


## Happy fitting!